In [ ]:
import sys, os
sys.path.insert(0, '../..')
# os.environ["CUDA_VISIBLE_DEVICES"]=""

os.environ['disable_tqdm'] = 'true'

In [ ]:
import numpy as np
import pandas as pd
import torch
from models.weighted_node2vec import WeightedNode2Vec
from dataset import triplet_dataset, pokec_data
from utils.config import DEVICE
import gc
from tqdm import trange
from utils.link_prediction import GCNLinkPrediction
import residual2vec as rv
import warnings
import faiss
import matplotlib.pyplot as plt
from utils.score import statistical_parity
from utils import graph_utils
from utils.snakemake_utils import FileResources
import seaborn as sns
from utils import snakemake_utils
import pickle as pkl
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
! ls /data/sg/ashutiwa/final_crosswalk/airport/airport_one/

In [ ]:
y = snakemake_utils.get_dataset("pokec").get_grouped_col()
sp_score, roc_score, gini_score, embs = {}, {}, {}, {}
DATA_ROOT = '/data/sg/ashutiwa/final_crosswalk/pokec/'
DIRS = ['pokec_one/', 'pokec_two/', 'pokec_three/', 'pokec_four/', 'pokec_five/']
DATASET = 'pokec'
model_to_embs = {
    "fairwalk+deepwalk": "_fairwalk_deepwalk.npy",
    "fairwalk+node2vec": "_fairwalk_node2vec.npy",
    "crosswalk+deepwalk": "_crosswalk_deepwalk.npy",
    "crosswalk+node2vec": "_crosswalk_node2vec.npy",
    "GCN+deepwalk+random": "_gcn_None_deepwalk_deepwalk_embs.npy",
    "GCN+deepwalk+r2v": "_gcn_None_deepwalk_r2v_embs.npy",
    "GCN+node2vec+random": "_gcn_None_node2vec_deepwalk_embs.npy",
    "GCN+node2vec+r2v": "_gcn_None_node2vec_r2v_embs.npy",
    "GAT+deepwalk+random": "_gat_None_deepwalk_deepwalk_embs.npy",
    "GAT+deepwalk+r2v": "_gat_None_deepwalk_r2v_embs.npy",
    "GAT+node2vec+random": "_gat_None_node2vec_deepwalk_embs.npy",
    "GAT+node2vec+r2v": "_gat_None_node2vec_r2v_embs.npy",
    "deepwalk": "_deepwalk_128_embs.npy",
    "node2vec": "_node2vec_128_embs.npy",
    "word2vec+deepwalk+random": "_word2vec_None_deepwalk_deepwalk_embs.npy"
    
}
M = list(model_to_embs.keys())

TEST_ADJ_FILE='/data/sg/ashutiwa/final_crosswalk/pokec/pokec_one/pokec_test_adj.npz'
NUM_NODES = 1632803
k=5

# distributions of positive and negative edgeswith dot product

In [ ]:
def get_roc_score(df):
    from sklearn.metrics import roc_auc_score
    y_pred = np.concatenate((df.positive_dot.values, df.negative_dot.values))
    y_true = np.concatenate((np.ones_like(df.positive_dot.values), np.zeros_like(df.negative_dot.values)))
    return roc_auc_score(y_true, y_pred)



In [ ]:
def plot_dot_(df, title):
#     df = pd.DataFrame({
#         "positive_dot": np.array([np.dot(a[i], p[i]) for i in trange(a.shape[0])]),
#         "negative_dot": np.array([np.dot(a[i], n[i]) for i in trange(a.shape[0])])
#     })
    
#     fig, ax = plt.subplots(1,1)
    df.positive_dot.plot(kind='density', title=title)
    df.negative_dot.plot(kind='density', title=title)
#     ax.set_xlim(-1000, 1000)
    ax.legend(['positive_neighbors', 'negative_neighbors'])
#     fig.show()

In [ ]:
FILE_CACHE = {}
def create_dot_df(A, file=TEST_ADJ_FILE):
    """
    A are node embeddings of shape # nodes \times 128
    """
    EDGE_IDX = snakemake_utils.get_edge_index_from_sparse_path(file)
    if file in FILE_CACHE:
        NEGATIVE_EDGE_IDX, EDGE_IDX = FILE_CACHE[file]
    else:
        NEGATIVE_EDGE_IDX, EDGE_IDX = graph_utils._negative_sampling_sparse(edge_index=EDGE_IDX, n_nodes=NUM_NODES, return_pos_samples=True)
        FILE_CACHE[file] = (NEGATIVE_EDGE_IDX, EDGE_IDX)
    print("selected edge_idx size: {}".format(EDGE_IDX.size(1)))

    positive_dot = np.sum(A[EDGE_IDX[0,:]] * A[EDGE_IDX[1,:]], axis=1)
    negative_dot = np.sum(A[NEGATIVE_EDGE_IDX[0,:]] * A[NEGATIVE_EDGE_IDX[1,:]], axis=1)
    dot = pd.DataFrame()
    dot["positive_dot"] = positive_dot
    dot["negative_dot"] = negative_dot
    
    return dot

# fairwalk+deepwalk

In [ ]:
# model = "fairwalk+deepwalk"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# fairwalk+node2vec

In [ ]:
# model = "fairwalk+node2vec"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape


In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# crosswalk+deepwalk

In [ ]:
# model = "crosswalk+deepwalk"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# crosswalk+node2vec

In [ ]:
# model = "crosswalk+node2vec"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GCN+deepwalk+random

In [ ]:
# model = "GCN+deepwalk+random"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GCN+deepwalk+r2v

In [ ]:
# model = "GCN+deepwalk+r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape


In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GCN+node2vec+random

In [ ]:
# model = "GCN+node2vec+random"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GCN+node2vec+r2v

In [ ]:
# model = "GCN+node2vec+r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT+deepwalk+random

In [ ]:
# model = "GAT+deepwalk+random"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT+deepwalk+r2v

In [ ]:
# model = "GAT+deepwalk+r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT+node2vec+random

In [ ]:
# model = "GAT+node2vec+random"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# GAT+node2vec+r2v

In [ ]:
# model = "GAT+node2vec+r2v"
# embs[model] = np.load(DATA_ROOT + DIRS[0] + DATASET + model_to_embs[model])
# A = embs[model]
# A.shape

In [ ]:
# # plot density
# dot = create_dot_df(A)
# fig, ax = plt.subplots(1,1)

# plot_dot_(dot, title=model + '_dot_density')
# # ax.set_xlim(-750, 750)
# ax.legend(['positive_neighbors', 'negative_neighbors'])
# fig.show()

In [ ]:
# # # create edge_list
# batch_size = 2000

# edges = snakemake_utils.return_new_graph(A, k, )
# gini_score[model] = statistical_parity(edges, y, metric='gini')
# sp_score[model] = statistical_parity(edges, y, )

# roc_score[model] = get_roc_score(dot)
# edges.shape, sp_score[model], roc_score[model], gini_score[model]

# plot statistical parity scores

# gini sp scores

In [ ]:

# for i, j in gini_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(gini_score.keys()), y=list(gini_score.values()))
# ax.set_ylim(min(gini_score.values()) * .9 , max(gini_score.values()) * 1.1)
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("gini statistical parity scores")

# sp scores

In [ ]:

# for i, j in sp_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(sp_score.keys()), y=list(sp_score.values()))
# ax.set_ylim(min(sp_score.values()) * .9, max(sp_score.values()) * 1.1)
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("statistical parity scores")

# roc auc scores

In [ ]:

# for i, j in roc_score.items(): print(i, j)
# sns.set(rc = {'figure.figsize':(15,8)})
# ax = sns.barplot(x=list(roc_score.keys()), y=list(roc_score.values()))
# ax.set_ylim(min(roc_score.values()) * .9, max(roc_score.values()) * 1.1)
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
# ax.set_title("roc scores")

In [ ]:

# N = 20

# sp_scores, gini_scores = np.zeros((N, len(M))), np.zeros((N, len(M)))
# for k in range(N):
#     for idx, m in enumerate(M):
#         if k == 0: continue
#         batch_size =  2000
#         A = embs[m]
#         edges = snakemake_utils.return_new_graph(A, k, )
        
    
#         gini_scores[k, idx] = statistical_parity(edges, y, metric='gini')
#         sp_scores[k, idx] = statistical_parity(edges, y, )


In [ ]:
# for i in range(len(M)):
#     plt.plot(gini_scores[:, i], label=M[i])
# plt.legend(loc="upper right")
# plt.title("gini scores vs # nn")

In [ ]:
# for i in range(len(M)):
#     plt.plot(sp_scores[:, i], label=M[i])
# plt.legend(loc="upper right")

# plt.title("sp scores vs # nn")

# combine all different runs

In [ ]:

def roc_scores(model_name):
    scores = np.zeros(len(DIRS))
    for idx, d in enumerate(DIRS):
        A = np.load(DATA_ROOT + d + DATASET + model_to_embs[model_name])
        file = DATA_ROOT + d + DATASET + '_test_adj.npz'
        dot = create_dot_df(A, file)
        scores[idx] = get_roc_score(dot)
    return scores


def statisticalp_scores(model_name, K=5):
    scores, scores_gini = np.zeros(len(DIRS)), np.zeros(len(DIRS))
    for idx, d in enumerate(DIRS):
        A = np.load(DATA_ROOT + d + DATASET + model_to_embs[model_name])
        edges = snakemake_utils.return_new_graph(A, K)
        scores_gini[idx] = statistical_parity(edges, y, metric='gini')
        
        scores[idx] = statistical_parity(edges, y, )
    return scores, scores_gini

In [ ]:

roc_score = {m: np.mean(roc_scores(m)) for m in model_to_embs}

for i, j in roc_score.items(): print(i, j)
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.barplot(x=list(roc_score.keys()), y=list(roc_score.values()))
ax.set_ylim(min(roc_score.values()) * .9, max(roc_score.values()) * 1.1)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
ax.set_title("roc scores")

In [ ]:

temp = {m: statisticalp_scores(m) for m in model_to_embs}

sp_score = {m: np.mean(v[0]) for m,v in temp.items()}
for i, j in sp_score.items(): print(i, j)
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.barplot(x=list(sp_score.keys()), y=list(sp_score.values()))
ax.set_ylim(min(sp_score.values()) * .9, max(sp_score.values()) * 1.1)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
ax.set_title("statistical parity scores")

In [ ]:
gini_score = {m: np.mean(v[1]) for m,v in temp.items()}
for i, j in gini_score.items(): print(i, j)
sns.set(rc = {'figure.figsize':(15,8)})
ax = sns.barplot(x=list(gini_score.keys()), y=list(gini_score.values()))
ax.set_ylim(min(gini_score.values()) * .9, max(gini_score.values()) * 1.1)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
ax.set_title("gini statistical parity scores")

In [ ]:

# roc_score, sp_score, gini_score= {}, {}, {}
roc_score = {m: roc_scores(m) for m in model_to_embs}
temp = {m: statisticalp_scores(m) for m in model_to_embs}
sp_score = {m: v[0] for m,v in temp.items()}
gini_score = {m: v[1] for m,v in temp.items()}
pkl.dump(roc_score, open(DATA_ROOT + "roc_score.pkl", 'wb'))

pkl.dump(sp_score, open(DATA_ROOT + "sp_score.pkl", 'wb'))

pkl.dump(gini_score, open(DATA_ROOT + "gini_score.pkl", 'wb'))